In [1]:
%cd image_retrieval/


/home/anlab/Downloads/imageclassification/image_retrieval


In [2]:
path_local="/home/anlab/Downloads/imageclassification/"

***Lấy file query ***

In [3]:

import os
def get_file_query():
  path= "gt_files/"
  list_gt = os.listdir(path)
  list_files_query=[]
  for file_name in list_gt:
    name=file_name.split(".")[0].split("_")[-1]
    if(name=="query"):
      list_files_query.append(file_name)
  return list_files_query



**lấy file ảnh tương ứng với querry (gồm 3 file good, oke, junk)**

In [4]:
#get file good,oke,junk correspond
def get_file_name(list_files_query, name):
  list_files_name=[]
  for query_file in list_files_query:
    name_correspond= query_file.replace("query",name)
    list_files_name.append(name_correspond)
  return list_files_name
  




**lấy ảnh trong file query**

In [5]:
def get_image_query(list_files_query):
  list_image_query=[]
  for query in list_files_query:
    dir= path_local+"/image_retrieval/gt_files/"+query
    with open(dir,"r") as f:
      for line in f.readlines():
        name_image= line.split(" ")[0].split("_",1)[1]+".jpg"
        list_image_query.append(name_image)
  return list_image_query

**lấy ảnh tương ứng của query với từng file good, oke, junk**

In [6]:


def get_image_correspond(file_name):
  list_image_name= []
  with open("gt_files/"+file_name,"r") as f:
    for line in f.readlines():
      list_image_name.append(line.split("\n")[0]+".jpg")
  return list_image_name

#Rmac


In [74]:
%cd keras_rmac/


/home/anlab/Downloads/imageclassification/image_retrieval/keras_rmac


In [117]:
from __future__ import division
from __future__ import print_function
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.layers import Lambda, Dense, TimeDistributed, Input
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
import keras
# from vgg16 import VGG16
from RoiPooling import RoiPooling
from PIL import Image,ImageOps
import scipy.io
import numpy as np
import utils
from torchvision import models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch
import cv2

def rzac(im_tensor, z='m', l0=1, L=3, ovr=0.4, norm=True, eps=1e-6, padding=0):
  N, C, H, W = im_tensor.size()
  w = np.minimum(W, H) # window size at scale 1
  fea = []
  for l in range(l0, l0+L):
    wl = int(np.floor(2*w/(l+1))) # window size at scale l
    wl = np.maximum(wl, 2)
    sl = int(np.floor((1-ovr)*wl)) # stride size at scale l
    sl = np.maximum(sl, 1)
    pl= padding if padding is not None else sl
    xl = F.max_pool2d(im_tensor,
                        kernel_size=(wl, wl),
                        stride=(sl, sl),
                        padding=(pl, pl))   
    fea.append(xl.view(N, C, -1))
    
  fea = torch.cat(fea, dim=2)
  fea = fea / (torch.norm(fea, p=2, dim=1, keepdim=True) + eps)
  fea = fea.transpose(1, 2) # (N, C, R ) -> (N, R, C)
  return fea


def get_feature(reg_feat_mat):
  reg_feat_mat = reg_feat_mat.squeeze() # [1, r, c] -> [r, c]
#   r = reg_feat_mat.size(0)
  ag_feat_vec = torch.sum(reg_feat_mat, dim=0, keepdim=False) # (r, c) -> (c,)
  ag_feat_vec = ag_feat_vec / (torch.norm(ag_feat_vec,
                                            p=2,
                                            dim=0,
                                            keepdim=True) + 1e-6)
  ag_feat_vec=np.array(ag_feat_vec).reshape(1,-1)
  return ag_feat_vec

In [118]:
#test 1 image
model= keras.applications.VGG16(include_top=False,weights=utils.DATA_DIR + utils.WEIGHTS_FILE)
file = utils.DATA_DIR + 'sample.jpg'
x = preprocessImage(file)
im_tensor = model.predict(x)
im_tensor=torch.tensor(im_tensor)
print("im_tensor = ",im_tensor.shape)

feature=rzac(im_tensor)
print("feature",feature.shape)

rmac=get_feature(feature)
print("rmac",rmac.shape)

im_tensor =  torch.Size([1, 512, 21, 32])
feature torch.Size([1, 12, 512])
rmac (1, 512)


In [20]:
def get_feature_Db(reg_feat_mat):
    reg_feat_mat = reg_feat_mat.squeeze() # [1, r, c] -> [r, c]
    return reg_feat_mat

In [21]:
def change_chanel(tensor_img):
    x= np.moveaxis(tensor_img, -1, 0)
    x=np.expand_dims(x, axis=0)
    return x

In [22]:
def preprocessImage(linkImage):
    img=cv2.imread(linkImage)
    x = change_chanel(img)
    return x

In [23]:
def get_Rmac(file,model):
    x = preprocessImage(file)
    #get feature each tensor image
    im_tensor = model.predict(x)
    #conver to torch tensor
    im_tensor=torch.tensor(im_tensor)
    #caculate rzac
    reg_feat_mat= rzac(im_tensor)   
    #caculate rmac 
    RMAC = get_feature(reg_feat_mat)
    return RMAC


In [30]:
def get_Rmac_DB(file,model):
    x = preprocessImage(file)
    #get feature each tensor image
    im_tensor = model.predict(x)
    #conver to torch tensor
    im_tensor=torch.tensor(im_tensor)
    #caculate rzac
    reg_feat_mat= rzac(im_tensor)
    nVector_represent=get_feature_Db(reg_feat_mat)
    return nVector_represent
    

In [25]:
%cd ..

/home/anlab/Downloads/imageclassification/image_retrieval


In [26]:
# get file query
list_files_query=get_file_query()
len(list_files_query)


55

In [27]:
%cd keras_rmac

/home/anlab/Downloads/imageclassification/image_retrieval/keras_rmac


In [28]:
# get file image correspond with query
im_list_query=get_image_query(list_files_query)
print(len(im_list_query))

55


In [29]:
from datetime import datetime
t1 = datetime.now().time()
from tqdm import tqdm
vector_query =[]
for img in tqdm(im_list_query):
    file = path_local+"Object Dataset/test/oxbuild_images/"+img
    # Load RMAC model
    RMAC=get_Rmac(file,model)
    vector_query.append(RMAC[-1])
t2 = datetime.now().time()

100%|██████████| 55/55 [00:25<00:00,  2.19it/s]


In [47]:
import pickle
from tqdm import tqdm
with open('dataset.pickle',"rb") as f:
    dataset = pickle.load(f)
dataset_new=[]
vector_dataset=[]
for img in tqdm(dataset):
    file = path_local+"Object Dataset/test/oxbuild_images/"+img
    # Load RMAC model
    RMAC_DB=get_Rmac_DB(file,model)
    vector_dataset.append(RMAC_DB)
    for i in range(RMAC_DB.shape[0]):
        dataset_new.append(img)

100%|██████████| 5063/5063 [12:30<00:00,  6.75it/s]


In [56]:
len(dataset_new)

53794

In [57]:
vector_dataset=np.vstack(vector_dataset)
print(vector_dataset.shape)


(53794, 512)


In [58]:

vector_query=np.array(vector_query)
print(vector_query.shape)

(55, 512)


In [59]:
# tính giao của 2 array-> kết quả trả về theo thứ tự của array 1
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [60]:
%cd /home/anlab/Downloads/imageclassification/image_retrieval/

/home/anlab/Downloads/imageclassification/image_retrieval


In [53]:
# %cd ..

In [61]:
list_files_good=get_file_name(list_files_query,"good")

In [67]:
topk=len(dataset_new)

In [63]:
from collections import OrderedDict
def map_name(list_files_query,name,vector_query,vector_dataset,dataset_new):
  list_ap_name=[]
  list_files_name=get_file_name(list_files_query,name)
  for i in range(len(list_files_query)):
    vector_l2_dataset=np.linalg.norm(vector_query[i]-vector_dataset,axis=-1)
    gt=get_image_correspond(list_files_name[i])
    toplen_image_query= np.argsort(vector_l2_dataset) 
    all_image_in_top = toplen_image_query[:topk]
    image_correspond = [dataset_new[image] for image in all_image_in_top]
    image_correspond = list(OrderedDict.fromkeys(image_correspond))
    index=[]
    AP=0
    for j in range(len(image_correspond)):
      if(image_correspond[j] in gt):
        index.append(j)
    for k in range(len(index)):
      AP= AP+ ((k+1)/(index[k]+1))
    AP=AP/len(gt)
    list_ap_name.append(AP)
  map_name=sum(list_ap_name)/len(list_ap_name)
  return map_name

In [64]:
map_good= map_name(list_files_query,"good",vector_query,vector_dataset,dataset_new)

In [65]:
print(map_good)


0.5806122069115497
